Nov 19
下面是找出来的几个候选  
pdfminer3k  
pdfminer.six  
pypdf  
pypdfocr 


In [1]:
# file paths
phone_numbers = ["./data/phone_numbers/070219_KV_BR_Telefonverzei.pdf",
                "./data/phone_numbers/52_pdf_wegweiser_schlaganfall.pdf",
                 "./data/phone_numbers/FL_SYB_BetriebsaerztlicherDienst_ID8414.pdf",
                 "./data/phone_numbers/Infoblatt_154.pdf"
                ]
scans = ["./data/scans/double_ocr.pdf",
         "./data/scans/single_ocr.pdf"]

isbn = ["./data/isbn/gelbe_seiten_2019-43.pdf"]

unit = ["./data/unit_conversion/si.txt"]


### 1-2 2 Methods of Converting PDF to Plain Text

### pdfminer.six
```
pip install pdfminer.six
```

In [82]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def convert(case, fname, pages=None):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);      
    manager = PDFResourceManager() 
    codec = 'utf-8'
    caching = True
    if case == 'text' :
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=LAParams())     
    if case == 'HTML' :
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    convertedPDF = output.getvalue()  
    infile.close(); converter.close(); output.close()
    return convertedPDF

#convertedPDF = convert('HTML', phone_numbers[0])
#print(convertedPDF)

### PDF TO Text
For Debian, Ubuntu, and friends
```
sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
```
For Fedora, Red Hat, and friends
```
sudo yum install gcc-c++ pkgconfig poppler-cpp-devel python3-devel
```
For Mac OS
```
brew install pkg-config poppler
```
then run
```
pip install pdftotext
```

In [64]:
import pdftotext

def get_text_pdftotext(pdf):
    with open(pdf, "rb") as f:
        pdf = pdftotext.PDF(f)
        return ("\n\n".join(pdf))
        #for page in pdf:
            #print(page)
print(get_text_pdftotext(phone_numbers[2]))


Betriebsärztlicher Dienst



Untersuchungen auf Wunsch des Beschäftigten                      Kontakt
Nach Arbeitsschutzgesetz hat jeder Arbeitgeber den Beschäf-
tigten eine arbeitsmedizinische Untersuchung zu ermöglichen,     Die Terminvergabe erfolgt
sofern der Mitarbeiter einen möglichen Zusammenhang              Montag – Donnerstag 8:00 – 16:00 Uhr
zwischen seiner Tätigkeit und seiner Erkrankung vermutet.        Freitag                8:00 – 15:00 Uhr
                                                                                                                                                                                                                                                   UniversitätsKlinikum Heidelberg
                                                                 zentral über die Telefonnummern
Verpflichtende Untersuchungen                                    06221 – 56 8966
Wer Tätigkeiten mit einer erhöhten Unfallgefährdung ausübt
oder solche, die mit dem Auf

## phone number format
### phone_numbers[0]
06221 / 56 8781  
06221 / 56 38099  
### phone_numbers[1]
0 62 21 – 56 78 42  en dash here  
0 72 61 – 66-0  
0 62 21 - 319 – 0  
0 72 61 – 19 29-2  
### phone_numbers[2]
06221 – 56 8966
### phone_numbers[3]
19292  
110  
56-7272  

前半段地区号： \+?\d+[[ ]?\d+]*[/|–|-]?[\s?\d+]*

### 1-2  4. Extract infomation from PDF

In [83]:
import re
re_email = re.compile("\w+@[\w-]+[\.\w+]+")
re_phone0 = r"(\d+ / (\d+([ ]\d+)*))"
re_phone1 = r"Tel\.:[ ]?(\d+[ \d+]*[ ]?[\-–]?[ ]?(\d+([ ]?[ \-–]?[ ]?\d+)*))"
re_phone2 = r"(\d+ [\-–] \d{2} \d{4})"
re_phone3 = r"Tel\. ((\d+\-?\d+)|(\d+ [\-–] \d{4,}))"
re_phone = re.compile(re_phone0 + '|' + re_phone1 + '|' + re_phone2 + '|' + re_phone3)
re_isbn = re.compile("\d*-\d*-\d*-\d*-\d*")
re_url = re.compile("https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}")

def extract_phone(regex, files):
    phones = []
    for file in files:
        text = get_text_pdftotext(file)
        infos = regex.findall(text)
        tuple_len = len(infos[0])
        for info in infos:
            for i in range(tuple_len):
                if info[i] != '':
                    phones.append(info[i])
                    break
    for num in phones:
        print(num)
    return phones
        

def extract_info(regex, files):
    for file in files:
        text = get_text_pdftotext(file)
        infos = regex.findall(text)
        #print(infos)
        for info in infos:
            print(info)
            
extract_phone(re_phone, phone_numbers)

06221 / 56 0
06221 / 56 6220
06221 / 56 8781
06221 / 56 8080
06221 / 56 7211
06221 / 56 6996
06221 / 56 7856
06221 / 56 7971
06221 / 56 7011
06221 / 56 2319
06221 / 56 4467
06221 / 56 6351
06221 / 56 4878
06221 / 56 38888
06221 / 56 38961
06221 / 56 38961
06221 / 56 38961
06221 / 56 6254
06221 / 56 6604
06221 / 56 6655
06221 / 56 6604
06221 / 56 6613
06221 / 56 6625
06221 / 56 2898
06221 / 56 6613
06221 / 56 6695
06221 / 56 6695
06221 / 56 6619
06221 / 56 6613
06221 / 56 6613
06221 / 56 6642
06221 / 56 4573
06221 / 56 6634
06221 / 56 4807
06221 / 56 6201
06221 / 56 4878
06221 / 56 6202
06221 / 56 6220
06221 / 56 6220
06221 / 56 6215
06221 / 56 6209
06221 / 56 7807
06221 / 56 6209
06221 / 56 6209
06221 / 56 36217
06221 / 56 6252
06221 / 56 6223
06221 / 56 6220
06221 / 56 6223
06221 / 56 38888
06221 / 56 6323
06221 / 56 6538
06221 / 56 36467
0160 / 90 86 03 72
06221 / 56 6224
06221 / 56 6225
06221 / 56 6281
0151 / 17408217
06221 / 56 39580
06221 / 56 6282
06221 / 56 6220
06221 / 56 6220


['06221 / 56 0',
 '06221 / 56 6220',
 '06221 / 56 8781',
 '06221 / 56 8080',
 '06221 / 56 7211',
 '06221 / 56 6996',
 '06221 / 56 7856',
 '06221 / 56 7971',
 '06221 / 56 7011',
 '06221 / 56 2319',
 '06221 / 56 4467',
 '06221 / 56 6351',
 '06221 / 56 4878',
 '06221 / 56 38888',
 '06221 / 56 38961',
 '06221 / 56 38961',
 '06221 / 56 38961',
 '06221 / 56 6254',
 '06221 / 56 6604',
 '06221 / 56 6655',
 '06221 / 56 6604',
 '06221 / 56 6613',
 '06221 / 56 6625',
 '06221 / 56 2898',
 '06221 / 56 6613',
 '06221 / 56 6695',
 '06221 / 56 6695',
 '06221 / 56 6619',
 '06221 / 56 6613',
 '06221 / 56 6613',
 '06221 / 56 6642',
 '06221 / 56 4573',
 '06221 / 56 6634',
 '06221 / 56 4807',
 '06221 / 56 6201',
 '06221 / 56 4878',
 '06221 / 56 6202',
 '06221 / 56 6220',
 '06221 / 56 6220',
 '06221 / 56 6215',
 '06221 / 56 6209',
 '06221 / 56 7807',
 '06221 / 56 6209',
 '06221 / 56 6209',
 '06221 / 56 36217',
 '06221 / 56 6252',
 '06221 / 56 6223',
 '06221 / 56 6220',
 '06221 / 56 6223',
 '06221 / 56 38888

#### 4.(i) From the PDF files in the folder phone_numbers/ extract as many valid phone numbers as possible

In [47]:
re_phone = re.compile("\+?\d+[\s?\d+]*[/|–|-]?[\s?\d+]*")
extract_info(re_phone, phone_numbers)

2007

6

                                                                   
8

                                                                   
10

                                                                   
14
                                                                   
26
                                                                   
32
                                                     
36
                                
40
                                                    
42
                                       
56                                        
106

114

116

72                                 
120

80
                                                                
122

82
                                                                
124

84

126

130
                                                                
92                          
132

06221 / 56 0



24 
06221 / 56 6220



06221 / 56 8781
                                  

19 222
        
32
69115 
69190 
2007


2003 
2004 
2005 
1
9
     
9
     
9
       
10
       
11
     
12
     
15
2
16
     
16
     
16
     
16
     
17
       
17
       
17
       
18
       
18
3
19
     
19
       
20
       
20
       
21
       
21
       
21
     
22
       
22
       
23
       
23
       
25
       
25
     
26
       
26
       
28
       
28
       
28
       
30
     
31


4
32
     
32
     
32
       
32
       
33
       
33
       
34
       
34
       
35
       
36
       
36
       
37
     
37
       
37
       
38
       
38
       
39
       
39
5
40
     
40
       
40
       
41
       
41
       
41
       
42
       
42
       
42
     
43
       
43
       
43
       
44
       
44
     
45
       
45
       
45
       
45
       
46
       
46
       
46
       
47
       
47
     
48


     
50
6
51
     
51
     
53
7
54
8
57
     
57
     
58
     
58
9
59
     
59
     
60
     
61
10
66
11
70
12
72
13
89


                     

#### 4.(ii) Extract valid URLs and Email addresses from the files in the folder phone_numbers/.

In [ ]:
# extract emails
re_email = re.compile("\w+@[\w-]+[\.\w+]+")
extract_info(re_email, phone_numbers)
# extract urls
re_url = re.compile("https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}")
extract_info(re_url, phone_numbers)

#### 4.(iii) From the PDF file in the folder isbn/ extract all valid ISBN numbers.

In [56]:
re_isbn = re.compile("\d*-\d*-\d*-\d*-\d*")
extract_info(re_isbn, isbn)

978-3-495-48431-9
978-3-85002-508-9
978-3-85002-536-2
978-3-85002-805-9
978-3-85002-839-4
978-3-99050-001-9
978-3-99050-045-3
978-3-9820330-0-6
978-3-9820330-1-3
978-3-401-06348-5
978-3-402-13429-0
978-3-945733-15-8
978-3-947266-01-2
978-3-947266-04-3
978-3-7633-5929-5
978-3-410-14201-0
978-3-410-14590-5
978-3-410-15459-4
978-3-410-15970-4
978-3-410-22634-5
978-3-410-61014-4
978-3-7106-0035-7
978-3-7106-0104-0
978-3-7106-0165-1
978-3-85033-968-1
978-3-7651-0485-5
978-3-406-37932-1
978-3-551-71470-1
978-3-532-62803-4
978-3-9813749-4-0
978-3-95619-090-2
978-3-7685-0948-0
978-3-7685-0968-8
978-3-7685-1253-4
978-3-7685-1353-1
978-3-7685-1553-5
978-3-7685-1753-9
978-3-7685-2353-0
978-3-7685-4144-2
978-3-7685-4753-6
978-3-7685-4853-3
978-3-7685-5053-6
978-3-7685-5353-7
978-3-7685-6144-0
978-3-7685-6644-5
978-3-7685-7144-9
978-3-7685-7744-1
978-3-7685-7944-5
978-3-7685-8744-0
978-3-7685-8944-4
978-3-7685-9305-2
978-3-7685-9505-6
978-3-7685-9805-7
978-3-438-04070-1
978-3-257-20057-7
978-3-257-

978-3-19-212927-8
978-3-19-212929-2
978-3-19-212951-3
978-3-19-212956-8
978-3-19-212957-5
978-3-19-212959-9
978-3-19-212964-3
978-3-19-212973-5
978-3-19-213307-7
978-3-19-213384-8
978-3-19-213385-5
978-3-19-214500-1
978-3-19-214504-9
978-3-19-214507-0
978-3-19-215272-6
978-3-19-215403-4
978-3-19-215466-9
978-3-19-217448-3
978-3-19-217918-1
978-3-19-217932-7
978-3-19-219475-7
978-3-19-219587-7
978-3-19-221080-8
978-3-19-221081-5
978-3-19-221083-9
978-3-19-221431-8
978-3-19-221655-8
978-3-19-221690-9
978-3-19-221704-3
978-3-19-221705-0
978-3-19-221741-8
978-3-19-221801-9
978-3-19-221806-4
978-3-19-221807-1
978-3-19-221808-8
978-3-19-221809-5
978-3-19-221816-3
978-3-19-221818-7
978-3-19-221825-5
978-3-19-222576-5
978-3-19-222884-1
978-3-19-222895-7
978-3-19-222917-6
978-3-19-222920-6
978-3-19-222927-5
978-3-19-222929-9
978-3-19-222951-0
978-3-19-222957-2
978-3-19-222958-9
978-3-19-222964-0
978-3-19-222966-4
978-3-19-222972-5
978-3-19-223307-4
978-3-19-223384-5
978-3-19-223385-2
978-3-19-2

978-3-86630-887-9
978-3-86630-893-0
978-3-86630-895-4
978-3-86630-896-1
978-3-86630-897-8
978-3-86630-899-2
978-3-86630-904-3
978-3-86630-905-0
978-3-86630-906-7
978-3-86630-907-4
978-3-86630-908-1
978-3-86630-909-8
978-3-86630-910-4
978-3-86630-911-1
978-3-86630-912-8
978-3-87870-171-2
978-3-87870-333-4
978-3-87870-335-8
978-3-87870-726-4
978-3-95786-220-4
978-3-935474-05-4
978-3-401-70143-1
978-3-455-00043-6
978-3-455-00120-4
978-3-455-00208-9
978-3-455-37010-2
978-3-455-37807-8
978-3-455-65011-2
978-3-455-65019-8
978-3-455-65022-8
978-3-455-65086-0
978-3-455-65140-9
978-3-455-65146-1
978-3-455-70022-0
978-3-86646-106-2
978-3-86646-135-2
978-3-411-74322-3
978-3-95958-047-2
978-3-95958-095-3
978-3-7667-1989-8
978-3-86244-667-4
978-3-95961-124-4
978-3-95961-125-1
978-3-95961-142-8
978-3-86230-349-6
978-3-86230-363-2
978-3-86230-369-4
978-3-8174-1588-5
978-3-8174-1908-1
978-3-932583-21-6
978-3-948036-01-0
978-3-649-62803-3
978-3-421-03817-3
978-3-421-03819-7
978-3-421-03833-3
978-3-421-

978-3-662-07193-9
978-3-662-07338-4
978-3-662-07342-1
978-3-662-07343-8
978-3-662-07492-3
978-3-662-07501-2
978-3-662-07588-3
978-3-662-07719-1
978-3-662-07776-4
978-3-662-07962-1
978-3-662-08070-2
978-3-662-08080-1
978-3-662-08172-3
978-3-662-08186-0
978-3-662-08207-2
978-3-662-08291-1
978-3-662-08293-5
978-3-662-08348-2
978-3-662-08362-8
978-3-662-08605-6
978-3-662-08639-1
978-3-662-08659-9
978-3-662-08913-2
978-3-662-08919-4
978-3-662-09174-6
978-3-662-09176-0
978-3-662-09178-4
978-3-662-09213-2
978-3-662-09223-1
978-3-662-09357-3
978-3-662-09391-7
978-3-662-09397-9
978-3-662-09418-1
978-3-662-09420-4
978-3-662-09422-8
978-3-662-09491-4
978-3-662-09587-4
978-3-662-09598-0
978-3-662-09762-5
978-3-662-09788-5
978-3-662-09790-8
978-3-662-09820-2
978-3-662-09864-6
978-3-662-10212-1
978-3-662-10566-5
978-3-662-10633-4
978-3-662-10708-9
978-3-662-10710-2
978-3-662-10888-8
978-3-662-10978-6
978-3-662-11173-4
978-3-662-12660-8
978-3-662-21758-0
978-3-662-22597-4
978-3-662-24805-8
978-3-662-

978-3-7091-6748-9
978-3-7091-6750-2
978-3-7091-6753-3
978-3-7091-6754-0
978-3-7091-6761-8
978-3-7091-6765-6
978-3-7091-6767-0
978-3-7091-6768-7
978-3-7091-6770-0
978-3-7091-6775-5
978-3-7091-6778-6
978-3-7091-6787-8
978-3-7091-6790-8
978-3-642-47709-6
978-3-642-47711-9
978-3-642-47713-3
978-3-642-50218-7
978-3-642-50231-6
978-3-642-51095-3
978-3-642-51100-4
978-3-642-51738-9
978-3-642-51851-5
978-3-642-52489-9
978-3-642-53728-8
978-3-642-53766-0
978-3-642-53767-7
978-3-642-53770-7
978-3-642-53780-6
978-3-642-53783-7
978-3-642-53797-4
978-3-642-53826-1
978-3-642-57329-3
978-3-642-57349-1
978-3-642-57352-1
978-3-642-57354-5
978-3-642-57355-2
978-3-642-57356-9
978-3-642-57357-6
978-3-642-57360-6
978-3-642-57361-3
978-3-642-57362-0
978-3-642-57363-7
978-3-642-57368-2
978-3-642-57369-9
978-3-642-57370-5
978-3-642-57371-2
978-3-642-57382-8
978-3-642-57383-5
978-3-642-57384-2
978-3-642-57385-9
978-3-642-57386-6
978-3-642-57389-7
978-3-642-57390-3
978-3-642-57391-0
978-3-642-57392-7
978-3-642-

#### 4.(iv) Convert values

In [80]:
def ml2l(ml_str):
    print(type(ml_str))
    # extract the number part
    ml_str = re.search("([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)", ml_str).group()
    # get rid of , in large numbers  "2,500" => "2500"
    ml_str = ml_str.replace(',', '')
    ml_num = float(ml_str)/1000
    return format(ml_num, '.3f')+" l"

f = open(unit[0],'r')
content = f.read()
print(content)
converted = re.sub(r"(([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)[ ]?(ml|milliliters|m))", ml2l, content)
print(converted)

1337 ml
2,500 ml
12 milliliters
26 l
18,421.9 ml
8321ml
32 m

<class 're.Match'>
<class 're.Match'>
<class 're.Match'>
<class 're.Match'>
<class 're.Match'>
<class 're.Match'>
1.337 l
2.500 l
0.012 l
26 l
18.422 l
8.321 l
0.032 l



In [38]:
a = re.compile("a")
print(type(a))

<class 're.Pattern'>
